In [1]:
import tensorflow as tf
import numpy as np

from tensorflow.keras import models, layers, losses, metrics, optimizers

# CNN Models

이번 튜토리얼에서는 CNN 모델을 빌드하는 법을 알아보겠습니다. 이전 튜토리얼에서는 sequential()로 빠르게 모델을 구성해봤는데, 이제부터 저만의 정석(?)대로 모델을 구성할 것입니다.

## 연습용 MNIST 데이터

먼저 연습용으로 쓸 MNIST 데이터를 로드합니다.

In [2]:
from tensorflow.keras.datasets.mnist import load_data

In [20]:
trainset, testset = load_data()

In [21]:
x_train, y_train = trainset
x_test, y_test = testset

In [22]:
print(x_train.shape, y_train.shape)
print(x_test.shape, y_test.shape)

(60000, 28, 28) (60000,)
(10000, 28, 28) (10000,)


In [23]:
x_train = x_train.reshape(*x_train.shape, 1)
x_test = x_test.reshape(*x_test.shape, 1)

다음은 간단히 데이터를 standardize하는 과정입니다.

In [24]:
# mean = 0, std = 0.5
x_train = (x_train.astype(np.float32) - 128) / 256
x_test = (x_test.astype(np.float32) - 128) / 256

다음으로는, 학습할 때 사용할 몇 가지 헬퍼 함수를 정의해보겠습니다.

In [7]:
def shuffle(x, y):
    """
    x, y를 셔플한다.
    
    Arguments:
    ----------
    x : features 데이터 행렬 (N, ...)
    y : 라벨 벡터            (N,)
    
    Returns:
    --------
    x[r] : x를 셔플한 np.array (N, ...)
    y[r] : y를 셔플한 np.array (N,)
    """
    
    n = x.shape[0]
    
    r = np.arange(n)
    np.random.shuffle(r)
    
    return x[r], y[r]

In [8]:
def next_batch(x, y, batch_size):
    """
    x, y를 해당 batch_size만큼 잘라서 배치를 생성해주는 generator
    
    Arguments:
    ----------
    x : features 데이터 행렬 (N, ...)
    y : 라벨 벡터            (N,)
    
    Returns:
    --------
    x_batch : x를 배치단위로 자른 것
    y_batch : y를 배치단위로 자른 것.
    """
    
    n = x.shape[0]
    n_batches = int(np.ceil(n / batch_size))
    
    for b in range(n_batches):
        start = b*batch_size
        end = min(n, (b+1)*batch_size)
        
        yield x[start:end], y[start:end]

자, 이제 모델을 구성할 것입니다.

제가 좋아하는 방식은 바로 subclassing 방식인데요, keras의 모델 클래스를 상속받아 저만의 모델을 만드는 것입니다! 왠지 객체지향적인 것 같아서 좋더라구요..
어쨌든, CNN 모델을 구성해보겠습니다.

In [9]:
class MyCNNModel(models.Model):
    """
    My CNN Model
    """
    
    def __init__(self):
        super(MyCNNModel, self).__init__() # 반드시 생성자 맨 처음에 호출해 줘야 함.
        
        # convolution - pool - convolution - pool
        self.features = models.Sequential([
            layers.Conv2D(8, (3, 3), strides=1, padding="same", input_shape=(28, 28, 1)),
            layers.BatchNormalization(),
            layers.Activation(tf.nn.tanh),
            
            layers.MaxPool2D((2, 2), strides=2, padding="same"),
            
            layers.Conv2D(12, (3, 3), strides=1, padding="same"),
            layers.BatchNormalization(),
            layers.Activation(tf.nn.tanh),
            
            layers.MaxPool2D((2, 2), strides=2, padding="same"),
        ])
        
        # dense - dense - dense
        self.classifier = models.Sequential([
            layers.Dense(64, input_shape=(7*7*12,)),
            layers.Activation(tf.nn.tanh),
            layers.Dropout(0.5),
            
            layers.Dense(32),
            layers.Activation(tf.nn.tanh),
            layers.Dropout(0.5),
            
            layers.Dense(10),
            layers.Activation(tf.nn.softmax)
        ])
        
    @tf.function
    def call(self, inputs, training=False):
        '''
        forward propagation function
        
        Arguments:
        ----------
        inputs : input tensor. (batch_size, 28, 28, 1)
        training: 트레이닝 과정이면 True 넣어줘야 함
        
        Returns:
        --------
        preds : 추론 결과
        '''
        
        # 하위 모델들도 본 모델과 같은 training을 적용시켜 준다.
        # (특히 dropout이 있는 경우 training=False일때 drop_rate = 0.0이 됨)
        self.features.training = training
        self.classifier.training = training
        
        n = tf.shape(inputs)[0]
        
        features = self.features(inputs)
        features = tf.reshape(features, (n, -1))
        
        preds = self.classifier(features)
        
        return preds

보시면, 의아해하실 수 있는게, 생성자에서 모델 구조를 두 개로 쪼개놨는데요, 이는 다음처럼 가운데에 flatten 레이어를 둬서 합칠 수도 있습니다.
```python
self.classifier = nn.Sequential([
    # features 구조
    
    layers.Flatten(),
    
    # classifier 구조
])
```

그런데, 왜 저렇게 했냐면, 보통 transfer learning을 할때, convolution 부분만 떼오고 classifier는 내것으로 새로 구현하는 경우가 상당히 많죠. 그래서, 저는 convolution 파트와 dense 파트를 분리해서 구현하는 것을 좋아합니다.

주의해야 할 점은 각 submodule(sequential 인 것들)의 첫 번째 레이어에 input_shape를 인자로 주시길 바랍니다.
또한, 그래프를 빌드하고 빠른 속도로 돌리기 위해 call 메소드는 @tf.function으로 데코레이팅 해 주시길 바랍니다. propagation이 빠르답니다.

In [10]:
mycnn = MyCNNModel()
mycnn.build(input_shape=(None, 28, 28, 1))

빌드했으니, summary를 볼 수 있습니다.

In [12]:
mycnn.summary()

Model: "my_cnn_model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
sequential (Sequential)      (None, 7, 7, 12)          1036      
_________________________________________________________________
sequential_1 (Sequential)    (None, 10)                40106     
Total params: 41,142
Trainable params: 41,102
Non-trainable params: 40
_________________________________________________________________


자 이제, optimizer, criterion, metric을 정의해 볼 텐데요, 새로운 것을 추가해보겠습니다.

In [13]:
optimizer = optimizers.Adam(learning_rate=1e-3)
criterion = losses.SparseCategoricalCrossentropy()

train_acc = metrics.SparseCategoricalAccuracy()
test_acc = metrics.SparseCategoricalAccuracy()

train_loss = metrics.Mean()
test_loss = metrics.Mean()

저 metric들은 compile할때 넣어주죠. 하지만, 저는 compile 및 fit을 사용하지 않을 것입니다. 좀 더 low level로 코딩할 것입니다.
저 metric들이 편리한 이유는, 안에 값을 축적해 뒀다가 result()를 호출하면 한번에 결과를 뱉어줍니다. 우리가 1 epoch 다 돌때까지 히스토리를 추적할 필요가 사라지죠.
원래, 배치 하나 돌고 loss 와 accuracy 저장하고, 이것을 1 epoch다 돌때까지 반복해서, 총 배치 개수로 나눠서 평균을 내주죠? 이것을 자동으로 해 준다고 생각하시면 됩니다.

다음은, 필요한 변수를 선언해봅니다.

In [14]:
EPOCHS = 10
BATCH_SIZE = 128

다음은, forward propagation을 위한 함수로, @tf.function으로 데코레이팅 해 주었습니다. 이는, 일반적인 파이썬 함수를 텐서플로 그래프로 빌드할 수 있게 해 주며, 파이썬 코드로 돌릴때보다 속도를 더 빠르게 해 줍니다.

In [16]:
@tf.function
def inference(model, criterion, x, y, training=False):
    """
    Forward propagation 함수.
    
    Arguments:
    ----------
    model : tf.keras 모델 객체
    criterion : loss 함수
    x : 데이터 x
    y : 라벨 y
    training : True이면 loss, predicitons, gradient를 계산해서 리턴해주고, False이면 loss와 prediction만 계산해서 리턴해준다.
    
    Returns:
    --------
    loss : 이 배치에 대한 loss
    preds : 이 배치에 대한 prediction 값
    [grads] : trainig=True일때 반환되며, 그래디언트를 계산한 것
    """
    
    if training is True:
        with tf.GradientTape() as tape:
            preds = model(x)
            loss = criterion(y, preds)
            
        grads = tape.gradient(loss, model.trainable_variables)
        return loss, preds, grads
    
    else:
        preds = model(x)
        loss = criterion(y, preds)
        return loss, preds

다음은, weight 업데이트 함수입니다. 역시 @tf.function으로 데코레이팅 해서 텐서플로 그래프로 만들어줍니다.

In [17]:
@tf.function
def backward(optimizer, grads, variables):
    """
    Backward propagation 함수
    
    Arguments:
    ----------
    optimizer : optimizer 객체
    grads : gradients list
    variables : weights들의 list
    """
    
    optimizer.apply_gradients(zip(grads, variables))

이제, 학습 루프를 구성해 봅시다. 사실 이 코드들은 fit()함수를 통해 간단하게 대체할 수 있지만, 나중에 복잡하고 커스텀 학습 방식을 구현할려면, fit을 사용하면 안됩니다.
미리미리 fit을 사용하지 않고 학습 루프를 구현하는 연습을 해 둘 필요가 있죠.

In [25]:
for e in range(EPOCHS):
    x_shuffled, y_shuffled = shuffle(x_train, y_train)
    
    for x_batch, y_batch in next_batch(x_shuffled, y_shuffled, BATCH_SIZE):
        loss, preds, grads = inference(mycnn, criterion, x_batch, y_batch, True)
        
        # 이것이, 결과를 축적해서 나중에 합쳐주는 metric입니다.
        train_loss(loss)
        train_acc(y_batch, preds)
        
        backward(optimizer, grads, mycnn.trainable_variables)
        
    for x_batch, y_batch in next_batch(x_test, y_test, BATCH_SIZE):
        loss, preds = inference(mycnn, criterion, x_batch, y_batch, False)
        
        test_loss(loss)
        test_acc(y_batch, preds)
        
    print(f"Epochs: {e+1}/{EPOCHS}")
    print(f"Train loss: {train_loss.result():.8f}")
    print(f"Train acc: {train_acc.result():.4f}")
    print(f"Test loss: {test_loss.result():.8f}")
    print(f"Test acc: {test_acc.result():.4f}")
    print()
    
    # 축적한 놈들을 없애버립니다.
    train_loss.reset_states()
    train_acc.reset_states()
    test_loss.reset_states()
    test_acc.reset_states()

Epochs: 1/10
Train loss: 0.37139958
Train acc: 0.9010
Test loss: 0.12006173
Test acc: 0.9638

Epochs: 2/10
Train loss: 0.10331572
Train acc: 0.9706
Test loss: 0.06877670
Test acc: 0.9781

Epochs: 3/10
Train loss: 0.07231963
Train acc: 0.9790
Test loss: 0.06409275
Test acc: 0.9788

Epochs: 4/10
Train loss: 0.05619738
Train acc: 0.9835
Test loss: 0.05313132
Test acc: 0.9835

Epochs: 5/10
Train loss: 0.04569481
Train acc: 0.9868
Test loss: 0.05213211
Test acc: 0.9828

Epochs: 6/10
Train loss: 0.03941837
Train acc: 0.9882
Test loss: 0.04398380
Test acc: 0.9864

Epochs: 7/10
Train loss: 0.03194543
Train acc: 0.9909
Test loss: 0.04153115
Test acc: 0.9861

Epochs: 8/10
Train loss: 0.02816586
Train acc: 0.9913
Test loss: 0.04224466
Test acc: 0.9850

Epochs: 9/10
Train loss: 0.02406918
Train acc: 0.9928
Test loss: 0.04003914
Test acc: 0.9866

Epochs: 10/10
Train loss: 0.02154790
Train acc: 0.9936
Test loss: 0.04245353
Test acc: 0.9856

